# README!

This notebook allows one to run the PHD filter code and visualize as well as process data for the MOT evaluation script to read.

To run Visualization you should at least have the data you care to run the filter on, properly formatted, and placed in the data folder.

Running the filter should produce data and place it in data/output. That output along with the associated images is needed to get the most out of this notebook.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

try: 
    from google.colab.patches import cv2_imshow
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError: 
    print("not using colab.")


not using colab.


# Setting up PHD Filter
This portion pulls and builds the filter code. If you have the filter code pulled, make sure to navigate "cd" into that directory. Uncomment out code below if needed

In [3]:
!git clone https://github.com/MobileRoboticistsW21/PHD-object-traking.git # don't need if cloned from terminal or etc

In [4]:
%cd PHD-object-traking/

In [5]:
!git checkout main   # can change this to main if branch is merged 
!git pull

In [6]:
%cd /content/PHD-object-traking/
!bash setup.sh

[sudo] password for akanu: 


### Notice!
The cell below runs the filter which will prompt you for the location of the data file you would like to process. Sample data is provided. Try ../data/first_20_frame_optical_res.json

In [6]:
!mkdir build
%cd build
!cmake ..
!make phd_json_dump
!./phd_json_dump
%cd ../

/home/akanu/git/PHD-object-traking/build
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so (found suitable version "3.8.5", minimum required is "3") 
-- Looking for sgemm_
-- Looking for sgemm_ - not found
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in p

# Visualization and Testing

If current directory is the that of the filter repository, all that needs to be set is the VIDEO_NAME and image_dir.

In [6]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import cv2
import numpy as np
import os 
import json

## Define Helper Functions

In [7]:
def load_data(detection_data_dir,filtered_data_dir,images_dir ):
    """
    detection_data_dir: JSON file generated from MASK RCNN
    filtered_data_dir: JSON file generated from PHD filter
    imaged_dir: should point to images in MOT-16 dataset (img)
    """
    try:
        with open(detection_data_dir, encoding='UTF-8', errors='ignore') as inputFile:
            data_input = json.load(inputFile)
    except: print("Cannot find sensor data!")

    try:
        with open(filtered_data_dir, encoding='UTF-8', errors='ignore') as inputFile:
            data_out = json.load(inputFile)
    except: print("Cannot find filtered data!")

    if images_dir:
        try:
            images = os.listdir(images_dir)
            images.sort()
            assert(len(images) > 0)
        except:
            print("No images were found!")
    return data_input, data_out, images

In [8]:
from python_utils.box_vis_utils import xywh_to_corners
    
def simple_filter_data_display(num_frames=1):
    for i in range(min(num_frames, len(data_out))):
        bbs = data_out[i]['bb']
        plt.clf(); plt.xlim(0, 1920); plt.ylim(0, 1080)
        ax = plt.axes()
        for bb in bbs:
            c = xywh_to_corners([bb])[0]
            rect = patches.Rectangle((c[0], c[1]), c[2], c[3], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        # plt.show()
        plt.pause(0.25)


In [9]:
# Miracle + Mingshuo's code to visualize Genertated & Mask RCNN Boxes with images.
from python_utils.box_vis_utils import xywh_to_corners

def overlay_boxes_on_images(num_frames = None, mode="save", path=None):
  '''
  mode can be "save" or "show"
  num_frames can be set to None To include all frames
  '''
  assert((mode == "save" and path != None) or mode == "show")
  for idx, (image_name,in_data, gen_data) in enumerate(zip(images[0:-2], data_input, data_out)):
    image = cv2.imread(os.path.join(images_dir, image_name))
    data_tlbr_in = xywh_to_corners(in_data['bb'])
    data_tlbr_gen = xywh_to_corners(gen_data['bb'])
    for bb in data_tlbr_in:
      alpha = 0.3
      overlay = image.copy()
      cv2.rectangle(overlay, (bb[0],bb[1]), (bb[2], bb[3]),(255,0,0), -1)
      cv2.addWeighted(overlay, alpha, image, 1 - alpha,0, image)
    for bb in data_tlbr_gen:
      cv2.rectangle(image, (bb[0],bb[1]), (bb[2], bb[3]),(0,255,0), 5)


    if mode == "show":
      # cv2.imshow(image_name, image)
      # cv2.waitKey(0) 
      # cv2.destroyAllWindows()
      plt.clf()
      plt.imshow(image)
      plt.pause(0.25)
    if mode == "save":
      print("\rSaving Image #",image_name, end='')
      cv2.imwrite((path + "/" + VIDEO_NAME + '_' + image_name), image)

    if num_frames and num_frames < idx: break 

In [10]:
# Combine image frames into a video with original resolution

def video_from_images(path):
  image_file_names = os.listdir(path)
  image_file_names = [x for x in image_file_names if x.startswith(VIDEO_NAME)]
  image_file_names.sort()
  assert(len(image_file_names) > 0)

  img = cv2.imread(os.path.join(path, image_file_names[0]))
  height,width,layers = img.shape

  fourcc = fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
  fps = 25

  video=cv2.VideoWriter(path + "/" + VIDEO_NAME+'_phd_filter_viz_video_short.mp4', \
                        fourcc, fps,(width,height))

  for img_name in image_file_names:
    img = cv2.imread(os.path.join(path, img_name))
    print("\rConstructing Video. Adding img: ", img_name, end='')
    video.write(img)
  cv2.destroyAllWindows()
  video.release()

## Set Data Directories & Visualize

To test and develop the two cells below can be re-run after data files are updated.

In [11]:
VIDEO_NAME = "MOT16-09"
images_dir = os.path.abspath("../" + VIDEO_NAME + "_images")
# images_dir = None # "/content/drive/My Drive/Mask_RCNN_stuff/" + VIDEO_NAME
filtered_data_dir = os.path.abspath("data/output/filtered_"+VIDEO_NAME+"_optical_flow_res.json")
detection_data_dir = os.path.abspath("data/" + VIDEO_NAME + "_optical_flow_res.json")
ground_truth_dir = ""

!mkdir data/output/temp_images_folder
output_img_dir = os.path.abspath("data/output/temp_images_folder")


In [12]:
## Load data
data_input, data_out, images = load_data(detection_data_dir,filtered_data_dir,images_dir)

## Display filter output when images are not available
# simple_filter_data_display(num_frames=2)

## Display images in notebook
# overlay_boxes_on_images(num_frames=5, mode="show", path=output_img_dir)

## Save images and make video
overlay_boxes_on_images(num_frames=None, mode="save", path=output_img_dir)
video_from_images(output_img_dir)

# Clean up images!
for f in os.listdir(output_img_dir):
    if f.endswith(".jpg"): os.remove(os.path.join(output_img_dir, f))


print("\nDone!")

Constructing Video. Adding img:  MOT16-09_000523.jpg
Done!


# Post processing and tuning

## Assigning IDs
In its current state the filter does not track IDs. Evaluation code for MOT expects IDs. This code is a workaround so we can run evaluation.

In [ ]:
from python_utils.id_assigner import assign_ids_postfact
data_with_ids_dir = 'data/output/filtered_' + VIDEO_NAME + '_with_ids'
data = assign_ids_postfact(data_out)
with open(data_with_ids_dir + '.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

## Formatting as txt
Adjusts text type and format to allow running the evaluation code provided by MOT

In [ ]:
from python_utils.json_to_txt import json_to_txt
json_to_txt(data_with_ids_dir)